In [1]:
import pandas as pd

# Задание 1

In [3]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


В ratings оставим только те строки, где rating =5. В полученном датафрейме сгрупируем фильмы по movieId и посчитаем для каждого movieId количество строк, то есть оценок 5. Отсортируем по полученному количестуву по убыванию. Возьмем первый фильм - это и есть фильм с наибольшим количеством пятерок

In [25]:
ratings[ratings.rating == 5].groupby('movieId').count().sort_values(by = 'userId', ascending=False)

,userId,rating,timestamp
movieId,,,
318,153,153,153
296,123,123,123
356,116,116,116
2571,109,109,109
260,104,104,104
...,...,...,...
6732,1,1,1
6731,1,1,1
6713,1,1,1


Найдем название фильма с movieId 318. Сюрприза нет: конечно, это "Побег из Шоушенка"

In [27]:
movies[movies.movieId==318]

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama


# Задание 2

In [95]:
power_df = pd.read_csv('power.csv')
power_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189482 entries, 0 to 1189481
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   country   1189482 non-null  object 
 1   year      1189482 non-null  int64  
 2   quantity  1189482 non-null  float64
 3   category  1189482 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 36.3+ MB


Создадим функцию для проверки условий, при соблюдений которых количество потребляемой энергии будет учитываться, а при не соблюдении - не будет. Фунцкция будет получать строку из датафрейма power_df и проверять попадает ли страна в перечень стран, категория потребления в перечень категорий, год в перечень годов. Кроме того, можно считать только положительные значения потребления или любые в зависимости от параметра positive_quantity only. Перечни могут быть списками, множестави и другим iterable

In [96]:
def filter_country_category_year (row, country_filter, category_filter, year_filter, positive_quantity_only = True):
    return True if (row['country'] in country_filter) and (row['category'] in category_filter) and (row['year'] in year_filter) and ((row['quantity'] > 0) or not positive_quantity_only) else False

Запишем ограничения из задачи в отдельные переменные. Чтобы их потом было удобно менять

In [97]:
baltic_states = {'Estonia', 'Latvia', 'Lithaunia'}
category_set = {4, 12, 21}
year_range = range(2005, 2011)

Применим с помощью apply нашу функцию к строкам датафрейма. В качестве списков будем передавать ограничения из условий задачи. Оставим только те строки, где функция принимает значение True. Сложим занчения потребляемой энергии quantity в этих строках

In [98]:
power_df[power_df.apply(filter_country_category_year, country_filter = baltic_states, category_filter = category_set, year_filter = year_range, positive_quantity_only = True, axis = 1) == True].quantity.sum()

118284.0

Можно было бы применять фильтры последовательно к датафрейму, записывая результат в отдельные колонки. Но мне кажется, реализованный способ лучше, так как короче. Интересно, где бы время вычислений было бы меньше